In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import math
import numpy as np
import random


In [2]:
def distance(x1, y1, x2, y2):
    """
    Calculates the Euclidean distance between two points.
    """
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)


In [3]:
def get_data_points(txt_file):
    ## Function to convert txt file into a pandas dataframe
    data = []
    with open(txt_file, "r") as file:
        lines = file.readlines()  # Read all lines in the file
        column_parsed = False
        column_names = []
        if column_parsed == False:
            words =  re.findall("\S+", lines[0])  # Extract column names from first line
            for word in words:
                column_names.append(word)   
            column_parsed=True                 
                
        for line in lines[1:]:  # Loop through lines, skipping the first line
            dataline = re.findall("\S+", line)  # Extract data points from line
            line_dict={}
            for i,point in enumerate(dataline):
                line_dict[column_names[i]]= float(point)  # Convert data point to float and store in dictionary
            data.append(line_dict)  # Append dictionary to list of data
    
    df = pd.DataFrame(data)   # Create pandas dataframe from list of dictionaries
    return df  # Return the dataframe



In [4]:
# Step 1: Initialize the data
customers = get_data_points("vrp data.txt")


In [19]:
def calculate_total_distance(customers, dist_matrix, routes):
    """
    Calculates the total distance of a set of routes
    
    Args:
        customers: A pandas dataframe representing customer data with the columns:
            CUST_NO, XCOORD, YCOORD, DEMAND, READY_TIME, DUE_DATE, SERVICE_TIME
        dist_matrix: A numpy array representing the distance matrix between customers
        routes: A list of routes, where each route is a list of customer numbers in the order they are visited
    
    Returns:
        The total distance of the routes
    """
    total_distance = 0
    for r in routes:
        distance = calculate_route_distance(customers, dist_matrix, r)
        total_distance += distance
    return total_distance


In [5]:
# Step 2: Calculate the distance matrix between all customers
    # You can use the euclidean distance for example

def calculate_distance_matrix(customers):
    """
    Calculates the distance matrix between all customers.

    Args:
        customers: A pandas dataframe with columns [CUST_NO, XCOORD, YCOORD, DEMAND, READY_TIME, DUE_DATE, SERVICE_TIME].

    Returns:
        A numpy array representing the distance matrix between all customers.
    """
    # Create an empty numpy array to store the distance matrix
    n_customers = len(customers)
    dist_matrix = np.zeros((n_customers, n_customers))

    # Loop over each pair of customers
    for i in range(n_customers):
        for j in range(n_customers):
            # Calculate the Euclidean distance between the XCOORD and YCOORD values of the two customers
            x_diff = customers['XCOORD'][i] - customers['XCOORD'][j]
            y_diff = customers['YCOORD'][i] - customers['YCOORD'][j]
            dist = math.sqrt(math.pow(x_diff, 2) + math.pow(y_diff, 2))

            # Store the calculated distance in the corresponding position in the distance matrix
            dist_matrix[i][j] = dist

    return dist_matrix


In [6]:
def check_vehicle_capacity(customers, new_route):
    """
    Checks if adding the new_route to the existing routes would violate the vehicle capacity constraint
    
    Args:
        customers: A list of tuples representing customer data in the format:
            (cust_no, xcoord, ycoord, demand, ready_time, due_date, service_time)
        new_route: A list of customer numbers in the order they are visited
        
    Returns:
        True if adding the new_route would not violate the vehicle capacity constraint, False otherwise
    """
    capacity = 0
    for cust in new_route:
        capacity += customers.loc[cust,"DEMAND"] # demand of customer
        if capacity > 200:
            return False
    return True


In [7]:
def check_time_windows(customers, new_route):
    def distance(point1, point2):
      x1, y1 = point1
      x2, y2 = point2
      return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    current_time = 0
    current_capacity = 0
    for i in range(len(new_route)):
        # Get the customer details
        cust = customers.loc[new_route[i]]
        xcoord, ycoord, demand, ready_time, due_date, service_time = cust[1:]
        
        # Calculate the travel time to the customer
        travel_time = distance((customers.loc[new_route[i-1], 'XCOORD'], customers.loc[new_route[i-1], 'YCOORD']), (xcoord, ycoord))
        
        # Calculate the time the customer will be serviced
        # service_start_time = max(current_time + travel_time, ready_time)
        service_start_time = max(current_time , ready_time)

        service_end_time = service_start_time + service_time
        
        # Check if the service time is within the customer's time window
        if service_end_time > due_date:
            return False
        
        # Update the current time and capacity
        current_time = service_end_time
        current_capacity += demand
        
        # Check if the capacity constraint is violated
        if current_capacity > 200:
            return False
    
    return True


In [8]:
def calculate_route_distance(customers, dist_matrix, route):
    """
    Calculates the total distance traveled on a given route
    
    Args:
        customers: A pandas dataframe representing customer data with the columns:
            CUST_NO, XCOORD, YCOORD, DEMAND, READY_TIME, DUE_DATE, SERVICE_TIME
        dist_matrix: A 2D numpy array representing the distance matrix between customers
        route: A list of customer numbers in the order they are visited
    
    Returns:
        The total distance traveled on the route
    """
    distance = 0
    for i in range(len(route) - 1):
        distance += dist_matrix[route[i], route[i+1]]
    distance += dist_matrix[route[-1], 0]
    return distance


In [9]:
def calculate_savings(customers, dist_matrix):
    """
    Calculates the savings for each pair of customers
    
    Args:
        customers: A pandas dataframe representing customer data with the columns:
            CUST_NO, XCOORD, YCOORD, DEMAND, READY_TIME, DUE_DATE, SERVICE_TIME
        dist_matrix: A 2D numpy array representing the distance matrix between customers
    
    Returns:
        A list of tuples representing the savings for each pair of customers, in descending order
    """
    # Calculate savings
    n_customers = len(customers)
    savings = []
    for i in range(2, n_customers):
        for j in range(i+1, n_customers):
            saving = dist_matrix[0][i] + dist_matrix[0][j] - dist_matrix[i][j]
            savings.append(((i,j), saving))
    
    # Sort savings in descending order
    sorted_savings = sorted(savings, key=lambda x: x[1], reverse=True)
    
    return sorted_savings


In [10]:
def calculate_dst(customers, dist_matrix):
    """
    Calculates the savings for each pair of customers
    
    Args:
        customers: A pandas dataframe representing customer data with the columns:
            CUST_NO, XCOORD, YCOORD, DEMAND, READY_TIME, DUE_DATE, SERVICE_TIME
        dist_matrix: A 2D numpy array representing the distance matrix between customers
    
    Returns:
        A list of tuples representing the savings for each pair of customers, in descending order
    """
    # Calculate savings
    n_customers = len(customers)
    dstances = []
    for i in range(2, n_customers):
        for j in range(i+1, n_customers):
            dst = dist_matrix[i][j]
            dstances.append(((i,j), dst))
    
    # Sort savings in descending order
    dstances = sorted(dstances, key=lambda x: x[1])
    
    return dstances


In [11]:
def get_vehicle_capacity(customers, new_route):
    """
    Checks if adding the new_route to the existing routes would violate the vehicle capacity constraint
    
    Args:
        customers: A list of tuples representing customer data in the format:
            (cust_no, xcoord, ycoord, demand, ready_time, due_date, service_time)
        new_route: A list of customer numbers in the order they are visited
        
    Returns:
        True if adding the new_route would not violate the vehicle capacity constraint, False otherwise
    """
    capacity = 0
    for cust in new_route:
        capacity += customers.loc[cust,"DEMAND"] # demand of customer
    return capacity


In [12]:
def clark_wright_savings(customers):
    """
    Applies Clark and Wright savings algorithm to generate initial solution
    
    Args:
        customers: A pandas dataframe representing customer data with the columns:
            CUST_NO, XCOORD, YCOORD, DEMAND, READY_TIME, DUE_DATE, SERVICE_TIME
    
    Returns:
        A list of routes, where each route is a list of customer numbers in the order they are visited
    """
    # Calculate distance matrix
    dist_matrix = calculate_distance_matrix(customers)
    
    # Calculate savings
    savings = calculate_savings(customers, dist_matrix)
    
    # Sort savings in descending order
    sorted_savings = sorted(savings, key=lambda x: x[1], reverse=True)

    # Initialize each customer as a separate route
    routes = [[i] for i in range(2, len(customers))]
    threshold = 10
    # Merge routes until vehicle capacity is maximized near its full capacity
    while all(get_vehicle_capacity(customers, r) < 200 - 10 for r in routes):
        
        merged = False
        for (i, j), s in sorted_savings:
            # Check if customers i and j are already in the same route
            i_route = None
            j_route = None
            for r in routes:
                if i in r:
                    i_route = r
                if j in r:
                    j_route = r
            if i_route == j_route:
                continue
                
            # Check if merging customers i and j would violate vehicle capacity or time windows
            new_route = i_route + j_route
            if check_vehicle_capacity(customers, new_route) and check_time_windows(customers, new_route):
                # Merge routes
                if i_route.index(i) == 0:
                    i_route.reverse()
                if j_route.index(j) == len(j_route) - 1:
                    j_route.reverse()
                i_route.extend(j_route)
                routes.remove(j_route)
                merged = True
        if not merged:
            break

    
    # Print routes and distances
    route_num = 1
    total_distance = 0
    new_routes = []
    for r in routes:
      if len(r)>=1:
        new_routes.append(r)

    for r in new_routes:
        route_str = f"Route {route_num} (Load {sum(customers.loc[r, 'DEMAND'])}): "
        route_str += " -> ".join([f"{int(c)}" for c in [1] + r + [1]])
        distance = calculate_route_distance(customers, dist_matrix, r)
        total_distance += distance
        route_str += f" (Distance {round(distance,2)})"
        route_str+= f" (load {round(get_vehicle_capacity(customers,r))})"
        print(route_str)
        route_num += 1
    
    # Print total distance and percentage of customers served
    print(f"Total distance: {round(total_distance,2)}")
    print(f"Percentage of customers served: {round(100 * len(set([c for r in new_routes for c in r])) / (len(customers)-2),2)}%")
      
    return new_routes


In [13]:
dist_matrix= calculate_distance_matrix(customers)

In [14]:
routes = clark_wright_savings(customers)

Route 1 (Load 47.0): 1 -> 37 -> 41 -> 15 -> 2 -> 4 -> 1 (Distance 95.94) (load 47)
Route 2 (Load 47.0): 1 -> 5 -> 17 -> 16 -> 1 (Distance 50.34) (load 47)
Route 3 (Load 26.0): 1 -> 6 -> 13 -> 1 (Distance 18.25) (load 26)
Route 4 (Load 30.0): 1 -> 8 -> 7 -> 10 -> 1 (Distance 51.84) (load 30)
Route 5 (Load 38.0): 1 -> 9 -> 35 -> 34 -> 1 (Distance 55.69) (load 38)
Route 6 (Load 44.0): 1 -> 11 -> 32 -> 20 -> 1 (Distance 57.92) (load 44)
Route 7 (Load 55.0): 1 -> 12 -> 79 -> 76 -> 1 (Distance 40.42) (load 55)
Route 8 (Load 34.0): 1 -> 19 -> 26 -> 1 (Distance 53.61) (load 34)
Route 9 (Load 27.0): 1 -> 73 -> 21 -> 57 -> 1 (Distance 41.62) (load 27)
Route 10 (Load 41.0): 1 -> 56 -> 23 -> 25 -> 1 (Distance 58.85) (load 41)
Route 11 (Load 36.0): 1 -> 69 -> 27 -> 53 -> 1 (Distance 20.97) (load 36)
Route 12 (Load 56.0): 1 -> 28 -> 50 -> 31 -> 1 (Distance 45.89) (load 56)
Route 13 (Load 48.0): 1 -> 78 -> 29 -> 68 -> 1 (Distance 40.36) (load 48)
Route 14 (Load 36.0): 1 -> 51 -> 30 -> 70 -> 1 (Distan

In [15]:
def nearest_neighbor(customers):

    # Calculate distance matrix
    dist_matrix = calculate_distance_matrix(customers)
    sorted_dst = calculate_dst(customers, dist_matrix)
    
    # Initialize each customer as a separate route
    routes = [[i] for i in range(2, len(customers))]
    threshold = 10
    # Merge routes until vehicle capacity is maximized near its full capacity
    while all(get_vehicle_capacity(customers, r) < 200 - 10 for r in routes):
        
        merged = False
        for (i, j), dst in sorted_dst:
            # Check if customers i and j are already in the same route
            i_route = None
            j_route = None
            for r in routes:
                if i in r:
                    i_route = r
                if j in r:
                    j_route = r
            if i_route == j_route:
                continue
                
            # Check if merging customers i and j would violate vehicle capacity or time windows
            new_route = i_route + j_route
            if check_vehicle_capacity(customers, new_route) and check_time_windows(customers, new_route):
                # Merge routes
                if i_route.index(i) == 0:
                    i_route.reverse()
                if j_route.index(j) == len(j_route) - 1:
                    j_route.reverse()
                i_route.extend(j_route)
                routes.remove(j_route)
                merged = True
        if not merged:
            break

    
    # Print routes and distances
    route_num = 1
    total_distance = 0
    new_routes = []
    for r in routes:
      if len(r)>=1:
        new_routes.append(r)

    for r in new_routes:
        route_str = f"Route {route_num} (Load {sum(customers.loc[r, 'DEMAND'])}): "
        route_str += " -> ".join([f"{int(c)}" for c in [1] + r + [1]])
        distance = calculate_route_distance(customers, dist_matrix, r)
        total_distance += distance
        route_str += f" (Distance {round(distance,2)})"
        route_str+= f" (load {round(get_vehicle_capacity(customers,r))})"
        print(route_str)
        route_num += 1
    
    # Print total distance and percentage of customers served
    print(f"Total distance: {round(total_distance,2)}")
    print(f"Percentage of customers served: {round(100 * len(set([c for r in new_routes for c in r])) / (len(customers)-2),2)}%")
      
    return new_routes


In [16]:
nearest_neighbor_routes = nearest_neighbor(customers)

Route 1 (Load 37.0): 1 -> 57 -> 2 -> 13 -> 1 (Distance 26.45) (load 37)
Route 2 (Load 36.0): 1 -> 84 -> 5 -> 60 -> 1 (Distance 26.62) (load 36)
Route 3 (Load 27.0): 1 -> 35 -> 9 -> 24 -> 1 (Distance 66.36) (load 27)
Route 4 (Load 56.0): 1 -> 11 -> 32 -> 10 -> 7 -> 1 (Distance 61.18) (load 56)
Route 5 (Load 61.0): 1 -> 12 -> 80 -> 68 -> 1 (Distance 29.7) (load 61)
Route 6 (Load 75.0): 1 -> 86 -> 44 -> 14 -> 17 -> 1 (Distance 66.49) (load 75)
Route 7 (Load 40.0): 1 -> 43 -> 15 -> 25 -> 4 -> 1 (Distance 80.36) (load 40)
Route 8 (Load 20.0): 1 -> 16 -> 91 -> 1 (Distance 30.71) (load 20)
Route 9 (Load 30.0): 1 -> 18 -> 89 -> 6 -> 1 (Distance 24.09) (load 30)
Route 10 (Load 56.0): 1 -> 49 -> 19 -> 20 -> 1 (Distance 74.08) (load 56)
Route 11 (Load 80.0): 1 -> 67 -> 23 -> 74 -> 22 -> 1 (Distance 54.13) (load 80)
Route 12 (Load 17.0): 1 -> 26 -> 1 (Distance 11.18) (load 17)
Route 13 (Load 48.0): 1 -> 27 -> 69 -> 70 -> 30 -> 1 (Distance 46.78) (load 48)
Route 14 (Load 32.0): 1 -> 53 -> 28 -> 55 

In [17]:
def objective_function(routes, customers, w1, w2, w3):
    # Calculate total vehicle load
    total_load = sum(get_vehicle_capacity(customers,route) for route in routes)
    # Calculate total number of vehicles
    total_vehicles = len(routes)
    # Calculate total distance traveled
    total_distance = calculate_total_distance(customers,dist_matrix,routes) 
    # Calculate weighted sum of objectives
    score = w1 * total_load - w2 * total_vehicles + w3 * total_distance
    return score


In [20]:
best_score = objective_function(routes, customers, 20, 10, 5)
best_score

35975.40200408809

In [21]:
def neighborhood_swap_customers(routes):
    """
    Generate a neighborhood of solutions by swapping two customers between routes.

    Args:
        routes (list): A list of routes, where each route is a list of customer numbers in the order they are visited.

    Returns:
        list: A list of solutions in the neighborhood, where each solution is a list of routes.
    """
    neighborhood = []
    for i in range(len(routes)):
        for j in range(i+1, len(routes)):
            for k in range(len(routes[i])):
                for l in range(len(routes[j])):
                    # Swap customers between routes
                    new_routes = [r.copy() for r in routes]
                    new_routes[i][k], new_routes[j][l] = new_routes[j][l], new_routes[i][k]
                    neighborhood.append(new_routes)
    return neighborhood


In [22]:
def evaluate_neighbors(current_solution, neighborhood, objective_function, customers, w1, w2, w3):
    neighbor_scores = {}
    for neighbor in neighborhood:
        neighbor_score = objective_function(neighbor, customers, w1, w2, w3)
        neighbor_scores[tuple(neighbor)] = neighbor_score
    current_score = objective_function(current_solution, customers, w1, w2, w3)
    return neighbor_scores, current_score


In [23]:
import random
import pandas as pd

# Define the objective function
def objective_function(routes, customers, w1, w2, w3):
    # Calculate total vehicle load
    total_load = sum(get_vehicle_capacity(customers,route) for route in routes)
    # Calculate total number of vehicles
    total_vehicles = len(routes)
    # Calculate total distance traveled
    total_distance = calculate_total_distance(customers,dist_matrix,routes) 
    # Calculate weighted sum of objectives
    score = w1 * total_load - w2 * total_vehicles + w3 * total_distance
    return score

# Define a function to generate a random initial solution
def generate_initial_solution(customers, capacity):
    # Shuffle the list of customers to create a random order
    shuffled_customers = list(customers['CUST_NO'])
    random.shuffle(shuffled_customers)
    # Initialize an empty list of routes
    routes = [[]]
    # Iterate over the shuffled customers and add them to routes
    for customer in shuffled_customers:
        route = routes[-1]
        if sum(customers.loc[customers['CUST_NO'].isin(route)]['DEMAND']) + customers.loc[customers['CUST_NO'] == customer]['DEMAND'].iloc[0] > capacity:
            # If the next customer would exceed the capacity, start a new route
            routes.append([customer])
        else:
            # Otherwise, add the customer to the current route
            route.append(customer)
    # Remove any empty routes
    routes = [route for route in routes if route]
    return routes

# Define a function to generate a neighborhood of solutions
def generate_neighborhood(solution):
    neighborhood = []
    for i in range(len(solution)):
        for j in range(len(solution[i])):
            for k in range(len(solution)):
                if k != i:
                    for l in range(len(solution[k])+1):
                        neighbor = [route[:] for route in solution]
                        customer = neighbor[i].pop(j)
                        neighbor[k].insert(l, customer)
                        neighborhood.append(neighbor)
    return neighborhood

# Define a function to evaluate the score of each solution in the neighborhood
def evaluate_neighborhood(neighborhood, customers, w1, w2, w3):
    scores = []
    for neighbor in neighborhood:
        score = objective_function(neighbor, customers, w1, w2, w3)
        scores.append(score)
    return scores

# Define a function to choose the best solution in the neighborhood
def choose_best_solution(solution, neighborhood, scores):
    current_score = objective_function(solution, customers, w1, w2, w3)
    best_score = min(scores)
    if best_score < current_score:
        best_index = scores.index(best_score)
        return neighborhood[best_index]
    else:
        return solution

# Define the main function that implements the local search algorithm
def local_search(customers, capacity, w1, w2, w3, max_iterations):
    # Generate a random initial solution
    solution = nearest_neighbor(customers)
    # Iterate over a fixed number of iterations
    for i in range(max_iterations):
        try:
            # Generate a neighborhood of solutions
            neighborhood = generate_neighborhood(solution)
            # Evaluate the score of each solution in the neighborhood
            scores = evaluate_neighborhood(neighborhood, customers, w1, w2, w3)
            # Choose the best solution in the neighborhood
        except:
            continue
        solution = choose_best_solution(solution, neighborhood, scores)
    # Return the best solution found
    return solution


In [24]:
sol = local_search(customers, 200, 500, 50, 10, 1)

Route 1 (Load 37.0): 1 -> 57 -> 2 -> 13 -> 1 (Distance 26.45) (load 37)
Route 2 (Load 36.0): 1 -> 84 -> 5 -> 60 -> 1 (Distance 26.62) (load 36)
Route 3 (Load 27.0): 1 -> 35 -> 9 -> 24 -> 1 (Distance 66.36) (load 27)
Route 4 (Load 56.0): 1 -> 11 -> 32 -> 10 -> 7 -> 1 (Distance 61.18) (load 56)
Route 5 (Load 61.0): 1 -> 12 -> 80 -> 68 -> 1 (Distance 29.7) (load 61)
Route 6 (Load 75.0): 1 -> 86 -> 44 -> 14 -> 17 -> 1 (Distance 66.49) (load 75)
Route 7 (Load 40.0): 1 -> 43 -> 15 -> 25 -> 4 -> 1 (Distance 80.36) (load 40)
Route 8 (Load 20.0): 1 -> 16 -> 91 -> 1 (Distance 30.71) (load 20)
Route 9 (Load 30.0): 1 -> 18 -> 89 -> 6 -> 1 (Distance 24.09) (load 30)
Route 10 (Load 56.0): 1 -> 49 -> 19 -> 20 -> 1 (Distance 74.08) (load 56)
Route 11 (Load 80.0): 1 -> 67 -> 23 -> 74 -> 22 -> 1 (Distance 54.13) (load 80)
Route 12 (Load 17.0): 1 -> 26 -> 1 (Distance 11.18) (load 17)
Route 13 (Load 48.0): 1 -> 27 -> 69 -> 70 -> 30 -> 1 (Distance 46.78) (load 48)
Route 14 (Load 32.0): 1 -> 53 -> 28 -> 55 

In [27]:
formulate_solution(sol)

Route 1 (Load 37.0): 1 -> 57 -> 2 -> 13 -> 1 (Distance 26.45)
Route 2 (Load 36.0): 1 -> 84 -> 5 -> 60 -> 1 (Distance 26.62)
Route 3 (Load 27.0): 1 -> 35 -> 9 -> 24 -> 1 (Distance 66.36)
Route 4 (Load 56.0): 1 -> 11 -> 32 -> 10 -> 7 -> 1 (Distance 61.18)
Route 5 (Load 61.0): 1 -> 12 -> 80 -> 68 -> 1 (Distance 29.7)
Route 6 (Load 75.0): 1 -> 86 -> 44 -> 14 -> 17 -> 1 (Distance 66.49)
Route 7 (Load 40.0): 1 -> 43 -> 15 -> 25 -> 4 -> 1 (Distance 80.36)
Route 8 (Load 20.0): 1 -> 16 -> 91 -> 1 (Distance 30.71)
Route 9 (Load 30.0): 1 -> 18 -> 89 -> 6 -> 1 (Distance 24.09)
Route 10 (Load 56.0): 1 -> 49 -> 19 -> 20 -> 1 (Distance 74.08)
Route 11 (Load 80.0): 1 -> 67 -> 23 -> 74 -> 22 -> 1 (Distance 54.13)
Route 12 (Load 17.0): 1 -> 26 -> 1 (Distance 11.18)
Route 13 (Load 48.0): 1 -> 27 -> 69 -> 70 -> 30 -> 1 (Distance 46.78)
Route 14 (Load 32.0): 1 -> 53 -> 28 -> 55 -> 1 (Distance 63.75)
Route 15 (Load 46.0): 1 -> 79 -> 29 -> 34 -> 1 (Distance 58.31)
Route 16 (Load 46.0): 1 -> 88 -> 31 -> 51 ->

In [ ]:
print(abdalla)

In [ ]:
import numpy as np
import pandas as pd
from typing import List

def insertion_heuristic(customers: pd.DataFrame, routes: List[List[int]], vehicle_capacity: int):
    """
    Improves the given solution using the insertion heuristic.
    """
    num_routes = len(routes)
    best_score = objective_function(routes, customers, 20, 10, 5)
    
    while True:
        improved = False
        
        for i in range(num_routes):
            route = routes[i]
            route_load = sum(customers.loc[customers['CUST_NO'].isin(route)]['DEMAND'])
            
            for j in range(len(route)-1):
                # select customer to insert
                cust_no = route[j+1]
                cust_load = customers.loc[customers['CUST_NO'] == cust_no]['DEMAND'].values[0]
                cust_x = customers.loc[customers['CUST_NO'] == cust_no]['XCOORD'].values[0]
                cust_y = customers.loc[customers['CUST_NO'] == cust_no]['YCOORD'].values[0]
                
                # try to insert customer into other routes
                for k in range(num_routes):
                    if k == i:
                        continue
                    
                    new_route = route.copy()
                    new_route.remove(cust_no)
                    new_route_load = route_load - cust_load
                    
                    # insert customer into new route
                    if len(routes[k]) > 2:
                        min_distance = np.inf
                        best_position = None
                        
                        for m in range(1, len(routes[k])):
                            prev_cust = routes[k][m-1]
                            next_cust = routes[k][m]
                            prev_cust_x = customers.loc[customers['CUST_NO'] == prev_cust]['XCOORD'].values[0]
                            prev_cust_y = customers.loc[customers['CUST_NO'] == prev_cust]['YCOORD'].values[0]
                            next_cust_x = customers.loc[customers['CUST_NO'] == next_cust]['XCOORD'].values[0]
                            next_cust_y = customers.loc[customers['CUST_NO'] == next_cust]['YCOORD'].values[0]
                            
                            distance1 = ((prev_cust_x - cust_x)**2 + (prev_cust_y - cust_y)**2)**0.5
                            distance2 = ((cust_x - next_cust_x)**2 + (cust_y - next_cust_y)**2)**0.5
                            distance3 = ((prev_cust_x - next_cust_x)**2 + (prev_cust_y - next_cust_y)**2)**0.5
                            
                            if new_route_load + cust_load <= vehicle_capacity and distance1 + distance2 - distance3 < min_distance:
                                min_distance = distance1 + distance2 - distance3
                                best_position = m
                                
                        if best_position is not None:
                            new_route.insert(best_position, cust_no)
                            routes[k] = routes[k][:1] + [cust_no] + routes[k][1:]
                            new_route_load += cust_load
                            
                            # check if solution improved
                            new_score = objective_function(routes, customers, 20, 10, 5)
                            if new_score < best_score:
                                routes[i] = new_route
                                best_score = new_score
                                improved = True
                                break
        
        # stop if no improvements can be made
        if not improved:
            break
            
    return routes, best_score


In [ ]:
new_routes,best_score = insertion_heuristic(customers,routes,200)

KeyboardInterrupt: 

In [ ]:
def calculate_total_distance(customers, dist_matrix, routes):
    """
    Calculates the total distance of a set of routes
    
    Args:
        customers: A pandas dataframe representing customer data with the columns:
            CUST_NO, XCOORD, YCOORD, DEMAND, READY_TIME, DUE_DATE, SERVICE_TIME
        dist_matrix: A numpy array representing the distance matrix between customers
        routes: A list of routes, where each route is a list of customer numbers in the order they are visited
    
    Returns:
        The total distance of the routes
    """
    total_distance = 0
    for r in routes:
        distance = calculate_route_distance(customers, dist_matrix, r)
        total_distance += distance
    return total_distance


It's possible that the genetic algorithm is not finding a solution where 100% of customers are served because the selection, crossover, and mutation operators are not tailored to the specific problem requirements. Here are a few suggestions to modify the code to increase the chances of finding a feasible solution:

Selection Operator: Instead of using tournament selection, you can try implementing a fitness proportionate selection method (e.g., roulette wheel selection) that biases the selection towards solutions that have a higher probability of being feasible.

Crossover Operator: Instead of using order crossover, you can try implementing a crossover method that ensures that the resulting offspring solutions have a higher probability of being feasible (e.g., edge recombination crossover).

Mutation Operator: Instead of using swap mutation, you can try implementing a mutation method that ensures that the resulting offspring solutions have a higher probability of being feasible (e.g., insertion or inversion mutation).

Fitness Function: You can modify the fitness function to include a penalty term that penalizes solutions that do not serve all customers. For example, you can add a penalty term that scales with the number of unserved customers or the total distance of the unserved customers.

By making these modifications, you can increase the chances of finding a feasible solution where 100% of customers are served.





In [ ]:
def all_customers_served(solution):
    """
    Checks if all customers have been served in a solution
    
    Args:
        solution: A list of routes, where each route is a list of customer numbers in the order they are visited
    
    Returns:
        True if all customers have been served, False otherwise
    """
    visited_customers = set()
    for route in solution:
        visited_customers.update(route)
    return visited_customers == set(range(len(customers)))


In [ ]:
def feasibility_func(population):
    feasible_population = [sol for sol in population if all_customers_served(sol) and check_vehicle_capacity(customers, sol) and check_time_windows(customers, sol)]
    return feasible_population


In [ ]:
def roulette_wheel_selection(population, fitness_func, feasibility_func, selection_size):
    # Filter out infeasible solutions
    
    feasible_population = feasibility_func(population)
    
    if not feasible_population:
        # No feasible solutions, return an empty list
        feasible_population = population # to make it continue for now #get to these again
    

    # Calculate the fitness of each feasible solution
    fitness_scores = [fitness_func(sol) for sol in feasible_population]
    total_fitness = sum(fitness_scores)
    
    # Calculate the probability of selecting each feasible solution
    probabilities = [score / total_fitness for score in fitness_scores]
    
    # Select solutions using roulette wheel selection
    selected = []
    for i in range(selection_size):
        r = random.uniform(0, 1)
        cumulative_probability = 0
        for j, sol in enumerate(feasible_population):
            cumulative_probability += probabilities[j]
            if cumulative_probability >= r:
                selected.append(sol)
                break
    
    return selected


In [ ]:
def repair_operator(customers,solution):
    repair_threshold = 100 
    # check if solution is feasible, and repair if necessary
    while not all(check_vehicle_capacity(customers, route) for route in solution ) or not all(check_time_windows(customers, route) for route in solution ):
        # randomly swap customers between routes
        route1 = random.randint(0, len(solution)-1)
        route2 = random.randint(0, len(solution)-1)
        if route1 == route2:
            continue
        
     
        customer1 = random.choice(solution[route1])
        customer2 = random.choice(solution[route2])

        solution[route1].remove(customer1)
        solution[route2].remove(customer2)
        solution[route1].append(customer2)
        solution[route2].append(customer1)
        repair_threshold-=1
        if repair_threshold ==0:
            break
    return solution


In [ ]:
def convert_to_tuple(obj):
    if isinstance(obj, list):
        return tuple(convert_to_tuple(item) for item in obj)
    else:
        return obj


In [ ]:
# function to get common customers 
def get_common_customers(parent1,parent2):
    common_customers= []
    for customer in parent1:
        if customer in parent2:
            if not customer in common_customers:
                common_customers.append(customer)
    for customer in parent2:
        if customer in parent1:
            if not customer in common_customers:
                common_customers.append(customer)

    return common_customers

In [ ]:
# Import random module
import random

# Define a function to create an edge map from two parent solutions
def create_edge_map(parent1, parent2):
    # Initialize an empty dictionary for the edge map
    edge_map = {}
    # Loop through each route in parent1
    for solution in parent1:
        for route in solution:
            # Loop through each customer in route
            for i, customer in enumerate(route):
                # Get the previous and next customer in route (wrap around if needed)
                prev_customer = route[i-1]
                next_customer = route[(i+1) % len(route)]
                # Add customer as a key and prev_customer and next_customer as values to edge map
                edge_map.setdefault(customer, set()).update([prev_customer, next_customer])
    # Repeat the same process for parent2
    for solution in parent2:
        for route in solution:
            for i, customer in enumerate(route):
                prev_customer = route[i-1]
                next_customer = route[(i+1) % len(route)]
                edge_map.setdefault(customer, set()).update([prev_customer, next_customer])
    # Return the edge map
    return edge_map

# Define a function to perform edge recombination crossover on two parent solutions
def edge_recombination_crossover(parent1, parent2):
    # Create an edge map from parent1 and parent2
    edge_map = create_edge_map(parent1, parent2)
    # Initialize an empty list for the offspring solution
    offspring = []
    # Initialize an empty set for the visited customers
    visited = set()
    # Choose a random starting customer from one of the parents
    current_customer = random.choice(random.choice(random.choice(parent1)))
    # Add current_customer to visited set
    visited.add(current_customer)

    while len(visited) < len(edge_map): 
        # Initialize an empty list for the current route 
        current_route = [] 
        # Add current_customer to current_route 
        current_route.append(current_customer) 
        while True: 
            # Remove current_customer from edge map and update all edges accordingly 
            edges = edge_map.pop(current_customer) 
            for neighbor in edges: 
                if neighbor in edge_map: 
                    edge_map[neighbor].remove(current_customer) 

                if not edges: 
                    break 

            try:
                input_of_min=[len(edge_map[neighbor]) for neighbor in edges if neighbor not in visited]
                print(input_of_min)
                min_edges = min(input_of_min) 
            except:
                min_edges=0    
            
            candidates = [neighbor for neighbor in edges if neighbor not in visited and len(edge_map[neighbor]) == min_edges] 

            
            if not candidates: 
                break 

            
            current_customer = random.choice(candidates) 

            
            visited.add(current_customer) 

            
            current_route.append(current_customer) 

        offspring.append(current_route) 

        remaining_customers = [customer for customer in edge_map.keys() if customer not in visited] 

        if not remaining_customers: 
            break 

        current_customer = random.choice(remaining_customers) 

        visited.add(current_customer)
    child = repair_operator(customers,offspring)

    return child


In [ ]:

def insertion_mutation(solution,feasibility_func):
    # randomly select two positions in the solution
    pos1 = random.randint(0, len(solution) - 1)
    pos2 = random.randint(0, len(solution) - 1)
    # insert the gene at pos1 to pos2
    gene = solution.pop(pos1)
    solution.insert(pos2, gene)
    try:
        feasible_solution = feasibility_func(mutated_population)[0]
    except:
        
        feasible_solution = solution

    return feasible_solution


In [ ]:
def count_unserved_customers(customers,solution):
    all_customers = set(range(len(customers)))
    served_customers = set(c for route in solution for c in route)
    unserved_customers = all_customers - served_customers
    return len(unserved_customers)


In [26]:
def formulate_solution(solution):
    route_num = 1
    total_distance = 0

    for r in solution:
        route_str = f"Route {route_num} (Load {sum(customers.loc[r, 'DEMAND'])}): "
        route_str += " -> ".join([f"{int(c)}" for c in [1] + r + [1]])
        distance = calculate_route_distance(customers, dist_matrix, r)
        total_distance += distance
        route_str += f" (Distance {round(distance,2)})"
        print(route_str)
        route_num += 1
        

    # Print total distance and percentage of customers served
    print(f"Total distance: {round(total_distance,2)}")
    print(f"Percentage of customers served: {round(100 * len(set([c for r in solution for c in r])) / (len(customers)-2),2)}%")
        


In [ ]:
import pandas as pd
import numpy as np

def calculate_capacity_penalty(customers, solution):
    # Calculate the total demand for each route in the solution
    demand_per_route = [sum(customers.loc[solution[i],'DEMAND']) for i in range(len(solution))]
    
    # Calculate the difference between the demand and capacity for each route
    capacity_diff = [demand_per_route[i] - 200 for i in range(len(demand_per_route))]
    
    # Apply a penalty for each route that exceeds the capacity
    capacity_penalty = sum([max(0, capacity_diff[i])**2 for i in range(len(capacity_diff))])
    
    return capacity_penalty


def calculate_time_window_penalty(customers, solution):
    # Calculate the arrival time for each customer in each route
    arrival_times = [[0] * (len(route) + 1) for route in solution]
    for i in range(len(solution)):
        route = solution[i]
        for j in range(1, len(route) + 1):
            prev_customer = route[j - 2] if j > 1 else 0
            curr_customer = route[j - 1]
            arrival_times[i][j] = max(arrival_times[i][j - 1], customers.loc[curr_customer, 'READY_TIME'])
            arrival_times[i][j] += customers.loc[curr_customer, 'SERVICE_TIME']
            # arrival_times[i][j] += customers.loc[curr_customer, 'DISTANCE'][prev_customer]
    
    # Calculate the difference between the arrival time and due date for each customer in each route
    time_diff = [[arrival_times[i][j] - customers.loc[solution[i][j-1], 'DUE_DATE'] for j in range(1, len(route) + 1)] for i, route in enumerate(solution)]
    
    # Apply a penalty for each customer that arrives after the due date
    time_penalty = sum([max(0, time_diff[i][j])**2 for i in range(len(time_diff)) for j in range(len(time_diff[i]))])
    
    return time_penalty


In [ ]:
def fitness_func(solution):
    total_distance = calculate_total_distance(customers, dist_matrix, solution)
    num_unserved_customers = count_unserved_customers(customers, solution)
    capacity_penalty = calculate_capacity_penalty(customers, solution)
    time_window_penalty = calculate_time_window_penalty(customers, solution)
    
    # Adjust penalty weights based on their importance
    penalty_weight = 1000
    capacity_penalty_weight = 3
    time_window_penalty_weight = 1
    
    # Calculate total penalty
    penalty_term = penalty_weight * (capacity_penalty_weight * capacity_penalty + time_window_penalty_weight * time_window_penalty)
    
    # Reward feasibility
    if num_unserved_customers == 0 and capacity_penalty == 0 and time_window_penalty == 0:
        bonus_term = 1.0
    else:
        bonus_term = 0.0
    
    # Return fitness value with penalty and bonus terms
    return 1.0 / (total_distance + penalty_term) + bonus_term


In [ ]:
# Step 1: get initial solution
routes = clark_wright_savings(customers)
dist_matrix = calculate_distance_matrix(customers)

# Step 2: Define genetic algorithm parameters
POPULATION_SIZE = 100
MAX_GENERATIONS = 20
FEASIBILITY_THRESHOLD = 10
NO_IMPROVEMENT_THRESHOLD = 2

# Step 3: Generate initial population
population = []
for i in range(POPULATION_SIZE):
    shuffled_routes = [random.sample(route, len(route)) for route in routes]
    population.append(shuffled_routes)


# Step 5: Implement selection, crossover, and mutation operators
best_fitness_value = float('-inf')
no_improvement_count = 0
for generation in range(MAX_GENERATIONS):
    # Step 5.1: Selection operator (e.g., roulette wheel selection)
    new_population = []
    for i in range(POPULATION_SIZE):
        parent1 = roulette_wheel_selection(population, fitness_func, feasibility_func, FEASIBILITY_THRESHOLD)
        parent2 = roulette_wheel_selection(population, fitness_func, feasibility_func, FEASIBILITY_THRESHOLD)
        child = edge_recombination_crossover(parent1, parent2)
        # Step 5.3: Mutation operator (e.g., insertion mutation)
        child = insertion_mutation(child, feasibility_func)

        new_population.append(child)
        
        # feasible_population = feasibility_func(new_population)
        if len(new_population) != 0:
            population = new_population
            # Step 6: Update termination criterion if feasible solution found
            best_solution = max(new_population, key=fitness_func)
            best_fitness_value = fitness_func(best_solution)
            no_improvement_count = 0
            print(f"Found feasible solution with fitness value {best_fitness_value}")
            break
        else:
            population = new_population
    
    # Step 6: Update termination criterion if no improvement in fitness value
    best_solution = max(population, key=fitness_func)
    formulate_solution(best_solution)
    new_best_fitness_value = fitness_func(best_solution)
    if new_best_fitness_value > best_fitness_value:
        best_fitness_value = new_best_fitness_value
        no_improvement_count = 0
    else:
        no_improvement_count += 1
        if no_improvement_count >= NO_IMPROVEMENT_THRESHOLD:
            print(f"No improvement in {NO_IMPROVEMENT_THRESHOLD} generations")
            break


Route 1 (Load 47.0): 1 -> 37 -> 41 -> 15 -> 2 -> 4 -> 1 (Distance 95.94)
Route 2 (Load 47.0): 1 -> 5 -> 17 -> 16 -> 1 (Distance 50.34)
Route 3 (Load 26.0): 1 -> 6 -> 13 -> 1 (Distance 18.25)
Route 4 (Load 30.0): 1 -> 8 -> 7 -> 10 -> 1 (Distance 51.84)
Route 5 (Load 38.0): 1 -> 9 -> 35 -> 34 -> 1 (Distance 55.69)
Route 6 (Load 44.0): 1 -> 11 -> 32 -> 20 -> 1 (Distance 57.92)
Route 7 (Load 55.0): 1 -> 12 -> 79 -> 76 -> 1 (Distance 40.42)
Route 8 (Load 34.0): 1 -> 19 -> 26 -> 1 (Distance 53.61)
Route 9 (Load 27.0): 1 -> 73 -> 21 -> 57 -> 1 (Distance 41.62)
Route 10 (Load 41.0): 1 -> 56 -> 23 -> 25 -> 1 (Distance 58.85)
Route 11 (Load 36.0): 1 -> 69 -> 27 -> 53 -> 1 (Distance 20.97)
Route 12 (Load 56.0): 1 -> 28 -> 50 -> 31 -> 1 (Distance 45.89)
Route 13 (Load 48.0): 1 -> 78 -> 29 -> 68 -> 1 (Distance 40.36)
Route 14 (Load 36.0): 1 -> 51 -> 30 -> 70 -> 1 (Distance 35.31)
Route 15 (Load 64.0): 1 -> 81 -> 33 -> 77 -> 3 -> 1 (Distance 37.02)
Route 16 (Load 35.0): 1 -> 36 -> 49 -> 1 (Distance 

KeyboardInterrupt: 

In [ ]:
best_solution

[[86, 35, 31],
 [45, 58, 88],
 [76, 40],
 [93, 27, 69],
 [50, 42, 79, 85, 47],
 [38, 87, 60],
 [24, 34, 6],
 [8, 54, 9]]

In [ ]:
formulate_solution(best_solution)

Route 1 (Load 70.0): 1 -> 86 -> 35 -> 31 -> 1 (Distance 127.44)
Route 2 (Load 43.0): 1 -> 45 -> 58 -> 88 -> 1 (Distance 79.4)
Route 3 (Load 22.0): 1 -> 76 -> 40 -> 1 (Distance 30.42)
Route 4 (Load 44.0): 1 -> 93 -> 27 -> 69 -> 1 (Distance 42.79)
Route 5 (Load 109.0): 1 -> 50 -> 42 -> 79 -> 85 -> 47 -> 1 (Distance 208.4)
Route 6 (Load 45.0): 1 -> 38 -> 87 -> 60 -> 1 (Distance 63.86)
Route 7 (Load 20.0): 1 -> 24 -> 34 -> 6 -> 1 (Distance 78.35)
Route 8 (Load 43.0): 1 -> 8 -> 54 -> 9 -> 1 (Distance 112.12)
Total distance: 742.78
Percentage of customers served: 25.25%


In [ ]:
def formulate_solution(solution):
    route_num = 1
    total_distance = 0

    for r in solution:
        route_str = f"Route {route_num} (Load {sum(customers.loc[r, 'DEMAND'])}): "
        route_str += " -> ".join([f"{int(c)}" for c in [1] + r + [1]])
        distance = calculate_route_distance(customers, dist_matrix, r)
        total_distance += distance
        route_str += f" (Distance {round(distance,2)})"
        print(route_str)
        route_num += 1
        

    # Print total distance and percentage of customers served
    print(f"Total distance: {round(total_distance,2)}")
    print(f"Percentage of customers served: {round(100 * len(set([c for r in solution for c in r])) / (len(customers)-2),2)}%")
        


In [ ]:
child

[[86, 35, 31],
 [45, 58, 88],
 [76, 40],
 [93, 27, 69],
 [50, 42, 79, 85, 47],
 [38, 87, 60],
 [24, 34, 6],
 [8, 54, 9]]

In [ ]:
formulate_solution(child)

Route 1 (Load 70.0): 1 -> 86 -> 35 -> 31 -> 1 (Distance 127.44)
Route 2 (Load 43.0): 1 -> 45 -> 58 -> 88 -> 1 (Distance 79.4)
Route 3 (Load 22.0): 1 -> 76 -> 40 -> 1 (Distance 30.42)
Route 4 (Load 44.0): 1 -> 93 -> 27 -> 69 -> 1 (Distance 42.79)
Route 5 (Load 109.0): 1 -> 50 -> 42 -> 79 -> 85 -> 47 -> 1 (Distance 208.4)
Route 6 (Load 45.0): 1 -> 38 -> 87 -> 60 -> 1 (Distance 63.86)
Route 7 (Load 20.0): 1 -> 24 -> 34 -> 6 -> 1 (Distance 78.35)
Route 8 (Load 43.0): 1 -> 8 -> 54 -> 9 -> 1 (Distance 112.12)
Total distance: 742.78
Percentage of customers served: 25.25%
